<a href="https://colab.research.google.com/github/rodianak/matsim-example-project/blob/master/Bachelor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following algorithms will be applied onto datasets, to analyze the employment of dependencies in data anonymization.

#Imports

In [ ]:
import pandas as pd
from os import uname
#from google.colab import drive
#drive.mount('/content/drive')
import math
import numpy as np
import itertools
from scipy.stats import entropy
import seaborn as sns
import pickle
import os
import random

###Loading of datasets and data cleansing

One of the datasets used for this work is the Adult dataset:
```

  author       = {Becker, Barry and Kohavi, Ronny},
  title        = {{Adult}},
  year         = {1996},
  howpublished = {UCI Machine Learning Repository},
  note         = {{DOI}: https://doi.org/10.24432/C5XW20}


```


After removing all the rows with missing values, the size of the dataset went from 48842 entries to 45221.

In [ ]:
path = "/content/drive/MyDrive/Datasets/adult.csv"

# inserting the names of the columns, for better readability
name_columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation",
                "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week",
                "native-country", "income"]

#adult = pd.read_csv(path, delimiter=";", header=None, names = name_columns)
""" get information from the dataset"""
#adult.info()

""" delete records with missing values"""
#adult.replace(' ?', np.nan, inplace=True)
#adult.dropna(inplace=True)

"""save cleaned dataset"""

#adult.to_csv("/content/drive/MyDrive/Datasets/adult_clean.csv", index=False)

adult = pd.read_csv("/content/drive/MyDrive/Datasets/adult_clean.csv")
#adult.info()
print(adult["hours-per-week"].unique().max())

# obesity dataset
obesityDataset = pd.read_csv("/content/drive/MyDrive/Datasets/ObesityDataSet.csv")
#obesityDataset.info()


99


173.0

#Computing L-Diversity

The algorithm "l_mondrian" is based on the multidimensional Mondrian algorithm for k-anonymity by LeFevre et. al.


```
  author={LeFevre, K. and DeWitt, D.J. and Ramakrishnan, R.},
  booktitle={22nd International Conference on Data Engineering (ICDE'06)},
  title={Mondrian Multidimensional K-Anonymity},
  year={2006},
  pages={25-25},
  keywords={Multidimensional systems;Protection;Greedy algorithms;Privacy;Approximation algorithms;Publishing;Demography;Public healthcare;Data security;Databases},
  doi={10.1109/ICDE.2006.101}

```



The hierarchies for the attributes workclass, education, marital-status and relationship are taken from the paper "Clustering ofmixed-type data considering concept hierarchies:
problem specification and algorithm", S.Behzadi et. al.

```

@article{d7acd3875c184273b4e295ff3f116d3f,
title = "Clustering of mixed-type data considering concept hierarchies: problem specification and algorithm",
keywords = "Information-theoretic clustering, Mixed-type data",
author = "Sahar Behzadi and M{\"u}ller, {N. S.} and Claudia Plant and C. B{\"o}hm",
note = "Publisher Copyright: {\textcopyright} 2020, The Author(s).",
year = "2020",
month = sep,
doi = "10.1007/s41060-020-00216-2",
language = "English",
volume = "10",
pages = "233--248",
journal = "International Journal of Data Science and Analytics",
issn = "2364-415X",
publisher = "Springer",
}

```



In [ ]:
# define generalization hierarchies for the categorical attributes

workclass_hierarchy = {
    "*": {
        "Self-employed": [' Self-emp-not-inc', ' Self-emp-inc'],
        "Government": [' Federal-gov', ' Local-gov', ' State-gov'],
        "Other": [' Private', ' Without-pay']
    }
}


education_hierarchy = {
    "*": {
        "High-school": [' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' 10th', ' 11th', ' 12th', ' HS-grad'],
        "undergrad": [' Bachelors', ' Some-college', ' Prof-school', ' Assoc-voc', ' Assoc-acdm', ' Preschool'],
        "academia": [' Masters', ' Doctorate']
    }
}


marital_status_hierarchy = {
    "*": {
        "married": [' Married-civ-spouse', ' Married-spouse-absent', ' Married-AF-spouse'],
        "single": [' Divorced', ' Separated', ' Widowed', ' Never-married']
    }
}

relationship_hierarchy = {
    "*": {
        "Family": [" Wife", " Own-child", " Husband", " Other-relative"],
        "No Family": [" Unmarried", " Not-in-family"]
    }
}
occupation_hierarchy = {
    "Occupation": {
        "White-Collar": [" Adm-clerical", " Exec-managerial", " Prof-specialty", " Sales", " Tech-support"],
        "Blue-Collar": [" Handlers-cleaners", " Other-service", " Transport-moving", " Farming-fishing",
                        " Machine-op-inspct", " Craft-repair", " Protective-serv", " Priv-house-serv", " Armed-Forces"]
    }
}

native_country_hierarchy = {
    "Native Country": {
        "North America": [" United-States", " Canada", " Outlying-US(Guam-USVI-etc)", " Puerto-Rico"],
        "Central/South America": [" Cuba", " Jamaica", " Mexico", " Honduras", " Columbia", " Dominican-Republic",
            " Ecuador", " El-Salvador", " Guatemala", " Haiti", " Peru", " Trinadad&Tobago", " Nicaragua"],
        "Europe": [" England", " Germany", " Poland", " Portugal", " France", " Italy", " Scotland", " Yugoslavia",
            " Greece", " Ireland", " Hungary", " Holand-Netherlands"],
        "Asia": [" India", " Iran", " Philippines", " Cambodia", " Thailand", " Laos", " Taiwan",
            " China", " South", " Japan", " Vietnam", " Hong"]
    }
}

income = {"income": [" <=50K", " >50K"]}

sex_hierarchy = {"*": [" Male", " Female"]}

"""
get hierarchy of an attribute
"""

def check_hierarchy(attribute):
  if attribute == 'workclass':
    return workclass_hierarchy
  elif attribute == 'education':
    return education_hierarchy
  elif attribute == 'marital-status':
    return marital_status_hierarchy
  elif attribute == 'relationship':
    return relationship_hierarchy
  elif attribute == 'occupation':
    return occupation_hierarchy
  elif attribute == 'native-country':
    return native_country_hierarchy
  elif attribute == "income":
    return income
  elif attribute == "sex":
    return sex_hierarchy
  elif attribute == "age":
    return age_hierarchy
  elif attribute == "fnlwgt":
    return fnlwgt_hierarchy
  elif attribute == "hours-per-week":
    return hours_per_week_hierarchy
  elif attribute == "capital-gain":
    return capital_gain_hierarchy
  elif attribute == "capital-loss":
    return capital_loss_hierarchy
  else:
    return None

"""
 defintion of helper functions
"""

# check, if dataset is l-diverse
def is_l_diverse(dataset, attribute, l):
  if dataset[attribute].nunique() >= l:
    return True
  else:
    return False

# find middle of the partition and return middle element
def find_middle(partition):

    middle_index = len(partition) // 2
    return partition[middle_index]

# return quasi identifier with the most unique values
def widest_qi(dataset, qis):
  unique_values_qi = []

  for qi in qis:
    unique_values_qi.append((qi, dataset[qi].unique()))
    sorted_qi = sorted(unique_values_qi, key=lambda x: len(x[1]), reverse=True)

  return sorted_qi[0][0]



# generalize categorical attribute to the next level in the generalization hierarchy
def gen_to_higher_level(attributeVal, hierarchy):

  for key, value in hierarchy.items():
    if isinstance(value, dict) and attributeVal in list(value.keys()):
        return key
    elif isinstance(value, dict):
      result = gen_to_higher_level(attributeVal, value)
      if result != None:
        return result
    elif attributeVal in value if isinstance(value, list) else attributeVal == value:
      return key
  return attributeVal

# take minimum(rounded down) and maximum (rounded up) value in numerical attribute and turn it into an interval
def generalize_numeric(dataset, qi):
  min_val, max_val = dataset[qi].min(), dataset[qi].max()
  if min_val == max_val:
    dataset[qi] = "*"
  elif qi == "age":
    min_val = min_val - min_val % 5
    max_val = max_val + 5 - (max_val % 5)
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "fnlwgt":
    min_val = min_val - min_val % 100000
    max_val = max_val + 100000 - (max_val % 100000)
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "hours-per-week":
    min_val = min_val - min_val % 5
    max_val = max_val + 5 - (max_val % 5)
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "capital-gain":
    min_val = min_val - min_val % 1000
    max_val = max_val + 1000 - (max_val % 1000)
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "capital-loss":
    min_val = min_val - min_val % 1000
    max_val = max_val + 1000 - (max_val % 1000)
    dataset[qi] = f"[{min_val} - {max_val}]"
  return dataset



def is_k_anonymous_categorical(partition, categorical_qis):

    # Select only the categorical quasi-identifier columns
    qi_data = partition[categorical_qis]

    # Count the occurrences of unique combinations of categorical QI values
    counts = qi_data.groupby(categorical_qis).size()

    # Check if all counts are greater than or equal to k
    # and explicitly return True or False
    k = len(partition)
    if (counts >= k).all():
        return True
    else:
        return False

def refine_generalization(dataset, qi):
# Work on a copy to avoid modifying the original

    #for qi in qis:
      #  if kind_of_attribute(qi) == "categorical":  # Numeric or categorical that has been generalized

  hierarchy = check_hierarchy(qi)
  if isinstance(hierarchy, list):
    print(f"Attribute: {qi}, Hierarchy Type: {type(hierarchy)}")
  if hierarchy is not None and isinstance(hierarchy, dict):
    dataset[qi] = dataset[qi].apply(lambda x: gen_to_higher_level(x, hierarchy))

  return dataset

# make partition k-anonymous
def ensure_k_anonymity_categorical(partition):
  categorical_qis = ['Gender', 'family_history_with_overweight', 'FAVC',  'CAEC', 'SMOKE','SCC', 'CALC', 'MTRANS']
  #["workclass", "education", "marital-status", "relationship", "occupation", "native-country"]

  generalized_partition = partition.copy()
  while not is_k_anonymous_categorical(generalized_partition, categorical_qis):
        # Find categorical columns with multiple values
        columns_to_generalize = [
            col for col in categorical_qis
            if generalized_partition[col].nunique() > 1  # More than one unique value
        ]

        if not columns_to_generalize:
            # If no columns can be generalized further, break the loop
            break

        # Generalize the first column in the list (minimal generalization)
        column_to_generalize = columns_to_generalize[0]
        hierarchy = check_hierarchy(column_to_generalize)

        generalized_partition[column_to_generalize] = generalized_partition[column_to_generalize].apply(lambda x: gen_to_higher_level(x, hierarchy))

  return generalized_partition


def generalize(dataset,qis):
  for qi in qis:
    if kind_of_attribute(qi) == "numeric":
      dataset = generalize_numeric(dataset, qi)
    else:
      dataset = refine_generalization(dataset, qi)

  dataset = ensure_k_anonymity_categorical(dataset)
  return dataset



"""
This recursive function checks for k-anonymity and l-diversity in each recursion
step. The algorithm is based on the algorithm of LeFevre et. al., as stated above.
"""

def l_mondrian(dataset, k, qis, l, sensitive_attribute):
    global recursive_call

    if len(dataset) < 2 * k:
      return generalize(dataset.copy(), qis)
    if recursive_call == 0:
      # picked age attribute first, for a more balanced split on this attribute
      selected_qi = "age"
    else:
      selected_qi = widest_qi(dataset, qis)

    # Handle numeric attributes
    if dataset[selected_qi].dtype == "int64":

        sorted_dataset = dataset.sort_values(by=selected_qi).copy()
        middle_index = len(sorted_dataset) // 2
        left_side = sorted_dataset.iloc[:middle_index].copy()
        right_side = sorted_dataset.iloc[middle_index:].copy()

    # Handle categorical attributes
    elif dataset[selected_qi].dtype == "object":
        print(selected_qi)
        hierarchy = check_obesity_hierarchy(selected_qi)

        dataset[selected_qi] = dataset[selected_qi].apply(lambda x: gen_to_higher_level(x, hierarchy))
        sorted_partition = sorted(dataset[selected_qi].unique())
        split_value = find_middle(sorted_partition)
        group_left = sorted_partition[:sorted_partition.index(split_value)]
        group_right = sorted_partition[sorted_partition.index(split_value):]
        left_side = dataset[dataset[selected_qi].isin(group_left)].copy()
        right_side = dataset[dataset[selected_qi].isin(group_right)].copy()

    # Recursively apply l_mondrian on the partitions
    if (len(left_side) >=k and is_l_diverse(left_side, sensitive_attribute, l)) and (len(right_side) >= k and is_l_diverse(right_side, sensitive_attribute, l)):
      recursive_call += 1
      partition_one = l_mondrian(left_side, k, qis, l, sensitive_attribute)
      partition_two = l_mondrian(right_side, k, qis, l, sensitive_attribute)


    else:
      return generalize(dataset.copy(), qis)


    # Concatenate the generalized partitions
    return pd.concat([partition_one, partition_two])


 # test with k=100

recursive_call = 0
test_path = "/content/drive/MyDrive/Datasets/adultsClean.csv"
test = pd.read_csv(test_path)
#mondrian = l_mondrian(test, 100, ["age", "workclass", "fnlwgt", "education", "marital-status","occupation", "relationship", "sex", "capital-gain","capital-loss", "hours-per-week",  "native-country"], 2, "income")
#mondrian[["age", "workclass","fnlwgt", "education", "marital-status","occupation", "relationship","sex", "hours-per-week",  "native-country", "income"]]


# Computing Relaxed Functional Dependencies

THe RFDs are computed with the DOMINO algorithm, as proposed in "Discovering Relaxed Functional Dependencies Based on Multi-Attribute Dominance", by L.Caruccio et. al.
```

  author={Caruccio, Loredana and Deufemia, Vincenzo and Naumann, Felix and Polese, Giuseppe},
  journal={IEEE Transactions on Knowledge and Data Engineering},
  title={Discovering Relaxed Functional Dependencies Based on Multi-Attribute Dominance},
  year={2021},
  volume={33},
  number={9},
  pages={3212-3228},
  keywords={Complexity theory;Approximation algorithms;Big Data;Distributed databases;Semantics;Lakes;Functional dependencies;data profiling;data cleansing},
  doi={10.1109/TKDE.2020.2967722}

```




In [ ]:
# helper function to create generalization hierarchies of numerical attributes

def create_gen_hierarchy(values, x):
    values = sorted(values)

    # This function creates the first level of intervals (first generalization level)
    def generate_intervals(values, x):
        # round minimum value down
        values[0]  = values[0] - values[0] % x
        # round maximum value up
        values[-1] = values[-1] + x - (values[-1] - values[0]) % x
        min_value = values[0]
        max_value = values[-1]

        # Start intervals from the minimum value and increase in steps of x
        intervals = []
        start = min_value
        while start < max_value:
            end = start + x
            # collect the interval
            intervals.append((start, min(end, max_value)))
            start += x

        return intervals

    # build the generalization hierarchy
    def build_hierarchy(values, x):
        hierarchy = []
        level = generate_intervals(values, x)
        hierarchy.append(level)

        # continue merging intervals until a single interval remains
        while len(level) > 1:
            # create the next level by merging adjacent intervals
            next_level = []
            for i in range(0, len(level), 2):
                if i + 1 < len(level):
                    merge = (level[i][0], level[i+1][1])
                    next_level.append(merge)

                else:
                    next_level.append(level[i])

            level = next_level
            hierarchy.append(level)

        return hierarchy


    return build_hierarchy(values, x)

# create hierarchies
age_values = test["age"].unique()
age_hierarchy = create_gen_hierarchy(age_values, 5)
fnlwgt_values = test["fnlwgt"].unique()
fnlwgt_hierarchy = create_gen_hierarchy(fnlwgt_values, 100000)
hours_per_week_values = test["hours-per-week"].unique()
hours_per_week_hierarchy = create_gen_hierarchy(hours_per_week_values, 5)
capital_gain_values = test["capital-gain"].unique()
capital_gain_hierarchy = create_gen_hierarchy(capital_gain_values, 1000)
capital_loss_values = test["capital-loss"].unique()
capital_loss_hierarchy = create_gen_hierarchy(capital_loss_values, 1000)


# compute after how many levels two numeric attribute values end up in the same interval
def generalization_distance_numeric(x,y, genHierarchy):
  if x == y:
    return 0
  else:
    height = 0
    interval_One = ()
    interval_Two = ()
    for level in genHierarchy:
      for interval in level:
        if x >= interval[0] and x <= interval[1]:
          interval_One = interval
        if y >= interval[0] and y <= interval[1]:
          interval_Two = interval
      height += 1
      if interval_One == interval_Two:
        break
  return height


# compute after how many levels two categoric attribute values end up in the same interval
def generalization_distance_categoric(valOne, valTwo, hierarchy):
  if valOne == valTwo:
    return 0
  else:
    height = 0
    keyOne = None
    keyTwo = None

    for key, value in hierarchy.items():

      if isinstance(value, dict):
          for inner_key, inner_value in value.items():
              if valOne in inner_value:
                  keyOne = inner_key
              if valTwo in inner_value:
                  keyTwo = inner_key

              if keyOne == keyTwo and keyOne is not None:
                  height += 1
                  break
              elif keyOne is not None and keyTwo is not None:
                  height +=1
                  break
      elif isinstance(value, list):
          if valOne in value and valTwo in value:
              height += 1
              break
          elif valOne in value or valTwo in value:
            height +=1
            break

      if keyOne is not None and keyTwo is not None and keyOne != keyTwo:
          return height + 1


    return height


########################### End of helper functions ############################


"""
The following functions implement the steps/algorithms mentioned in the paper.
"""

############################### Step 3.1: ######################################

#Pairwise computation of the distance vectors between entries in given dataframe



distance_data = []


# create distance matrix with generalization distance as metric
def compute_delta_df(dataset, num_rows, qi_columns, sensitive_attribute):
  for i in range(num_rows):
    for j in range(i + 1, num_rows):
        row_distances = {}
        for column in dataset.columns:
            if column in qi_columns:
                if dataset[column].dtype == "int64":
                    distance = generalization_distance_numeric(dataset.loc[i, column], dataset.loc[j, column], check_hierarchy(column))
                else:
                    distance = generalization_distance_categoric(dataset.loc[i, column], dataset.loc[j, column], check_hierarchy(column))
                row_distances[column] = distance
            else:
                row_distances[column] = None
        distance_data.append(row_distances)
  distance_df = pd.DataFrame(distance_data)
  return distance_df.sort_values(by=sensitive_attribute)



# test
testDM = pd.read_csv("/content/drive/MyDrive/Datasets/adultDM.csv")
sample_size = 500
testDM_sample = testDM.head(sample_size)
test_rows = len(testDM_sample)


test_columns = ["age", "workclass", "fnlwgt", "education", "marital-status", "occupation",
                "relationship", "sex", "capital-gain", "capital-loss",  "hours-per-week",
                "native-country", "income"]

#testDelta = compute_delta_df(testDM_sample, test_rows, test_columns, "income")


# save for later use
#with open('/content/drive/MyDrive/pickledData/btestDelta.pickle', 'wb') as f:
 #   pickle.dump(testDelta, f)


#display(testDelta.head(100))
################################ Step 3.2 ######################################

# Constructing T_beta relations.
# T_beta contains all distance vectors that to not dominate each other with
# sensitive_attribute values bigger than beta.


# check, if vector v dominates any of the vectors in matrix T
def dominates(v, T):
    for t in T:
      if (v >= t).all() and (v > t).any():
        return True
      else:
        return False

# Remove vectors t from T that dominate w
def remove_dominated(w, T):
    return [t for t in T if not ((t >= w).all() and (t > w).any())]


# Create T_beta relations as described in the paper (Algorithm 2)
def create_T_relations(df, S):
  T = []
  T_beta = []
  beta = df.iloc[0][S]
  beta_values = []
  beta_values.append(beta)
  sorted_df = df.sort_values(by=S, ascending=False)
  for index, row in sorted_df.iterrows():
    w = row.values

    if w[-1] not in beta_values:
      beta = w[-1]
      beta_values.append(beta)
      T.append(T_beta)
      T_beta = []

    if not dominates(w, T_beta):
      T_beta.append(w)
      remove_dominated(w, T_beta)

  T.append(T_beta)

  return T


################################# Step 3.3 #####################################

# Generating LHS candidates

# compute difference matrix of vector w in T_beta
def difference_matrix(T_beta, w):
  DM = []
  for v in T_beta:
    if not np.array_equal(v, w):
      difference = np.subtract(w,v)
      # removing the sensitive attribute, since it will not be needed in the DM
      DM.append(difference[:-1])
  return DM

# check, if given column combination X is a viable candidate (Definition 7)
def is_viable(DM, X):
    for row in DM:
        is_admissible = False

        for index in X:
            if row[index] < 0:
                is_admissible = True
                break

        if not is_admissible:
            all_zeros = True
            for index in X:
                if row[index] != 0:
                    all_zeros = False
                    break
            if not all_zeros:
                # If not all zeros and no negative value, row is not admissible

                return False

    # If all rows are admissible, return True
    return True



#Algorithm 3 in Paper
def create_lhs_candidates(T_beta, w):
  L = 1
  levelAttr = []
  LHS = set()

  DM = difference_matrix(T_beta, w)
  num_attributes = len(w) - 1

  while L <= num_attributes:
    # generate column combinations
    levelAttr = list(itertools.combinations(range(num_attributes), L))

    new_levelAttr = []
    # iterate over all combinations (sub-vectors) of size L
    for X in levelAttr:
      # Check if the length of the combination is less than or equal to 4 before checking viability
      if len(X) <= 4 and is_viable(DM,X):
        LHS.add(tuple(X))  # Add the sub-vector X to LHS if viable and length is within limit

    L += 1

  filtered_lhs = {tuple for tuple in LHS if len(tuple)>1}
  return filtered_lhs

################################ Step 3.4 ######################################

# Determining threshold values of minimal RFDcs


def find_min(TB_relation, m_i, sk, i,  j, thrs, w):
    """
    Args:
        TB_relation: The T_B relation (list of distance vectors).
        m_j: The current value for the distance vector being evaluated (corresponding to attribute j).
        sk: The current distance vector (LHS combination being evaluated).
        i: The index of the attribute being evaluated.
        thrs: The current list of thresholds.
    Returns:
        The minimum valid threshold value for the attribute X_j, or -1 if no valid value is found.
    """
    # initialize with a large number to find the minimum valid threshold
    min_value = float('inf')
    for row in TB_relation:
      if not np.array_equal(row, w) and j != i:
        p_j = row[sk[j]]

        # Step 1 and 2: p_j > d_j and p_i < d_i
        if p_j <= w[sk[j]] and row[sk[i]] >= m_i:
            continue  # Skip this row if p_j <= m_j

        # Assume the row is valid unless proven otherwise
        valid_row = True


        # Step 3: all previous attributes satisfy their thresholds
        for e in range(j):
          if e==i:
            continue
          elif thrs[e] is not None and row[sk[e]] > thrs[e]:
              valid_row = False
              break

        if not valid_row:
            continue
        valid_column = True

        # Step 4: Check remaining attributes after j
        for e in range(j + 1, len(sk)):
          if not any(lst[e] >= row[e] for lst in TB_relation):
            valid_column = False
        if valid_row and valid_column:
            min_value = min(min_value, p_j)
            break

    # Return the minimal valid value or -1 if no valid value is found
    return min_value if min_value != float('inf') and min_value > 1 else -1


# Helper functions
def get_vector(w, cc):
    return [i for i in cc]

#Create the RFDc based on the given thresholds and their column combinations cc
def create_rfdc(thrs, cc):
    return {"thresholds": thrs, "cc": cc}



# Algorithm 4 in paper
def determine_thresholds(TB_relation, w, LHS):
    """
    Args:
        TB_relation: List of distance vectors.
        w: A distance vector w in TB_relation.
        LHS: Set of viable column combinations.
    Returns:
        List of discovered RFDcs.
    """
    admissible = False
    RFDcs = []

    # iterate over all LHS combinations
    for cc in LHS:
      # get indices of distance vector for the current LHS combination
        sk = get_vector(w, cc)

      # get values of the indices in sk
        check_lhs = []
        for k in sk:
          check_lhs.append(w[k])

      # values of LHS thresholds are not allowed to equal 0
        if not all(x > 0 for x in check_lhs):
          continue

        thrs = []

        # Loop through the columns of sk (LHS combination)
        for i in range(len(sk)):
            m_i = w[sk[i]]

            if m_i > 1:
              admissible = True
              thrs = [None] * len(sk)
              # rule for epsilon step of 1
              thrs[i] = (m_i - 1)
              for j in range(len(sk)):
                if i == 0:
                  j +=1
                min_thrs = find_min(TB_relation, m_i, sk, i, j, thrs, w)

                if min_thrs != -1:
                  thrs[j] = (min_thrs - 1)

                  if all(alpha != None for alpha in thrs):
                    break
                else:

                  admissible = False
                  break

        # If admissible, create RFD and add it to the list of RFDs.
        if admissible:
          new_RFDc = create_rfdc(thrs, cc)
          if new_RFDc in RFDcs:
            RFDcs.remove(new_RFDc)
          RFDcs.append(new_RFDc)

    return RFDcs


###################################Test#########################################

# load difference relation from delta(r) from the previous run

with open('/content/drive/MyDrive/pickledData/btestDelta.pickle', 'rb') as f:
  testDelta = pickle.load(f)

testDelta_reduced = testDelta[["age", "workclass", "fnlwgt", "education", "marital-status", "occupation",
                "relationship", "sex", "capital-gain", "capital-loss", "hours-per-week",
                "native-country", "income"]]
#print(len(testDelta))
#testDelta_reduced.head(100)
# create T_beta
#t_zero = create_T_relations(testDelta_reduced, "income")


with open('/content/drive/MyDrive/pickledData/btzero.pickle', 'rb') as f:
    t_zero = pickle.load(f)

#print(t_zero[1])
#print(len(t_zero[1]))


with open('/content/drive/MyDrive/pickledData/vectorSample.pickle', 'rb') as f:
    sample_vectors = pickle.load(f)


# get all viable column combinations for every vector in t_zero
def all_lhs_tbeta(T_beta):
  lhs_candidates_list = []

  for w in T_beta:
    lhs_candidates = create_lhs_candidates(T_beta, w)
    lhs_candidates_list.append(lhs_candidates)
  return lhs_candidates_list

#adult_lhs_candidates = all_lhs_tbeta(sample_vectors)

#with open('/content/drive/MyDrive/pickledData/btest_lhs_candidates.pickle', 'wb') as f:
 #   pickle.dump(adult_lhs_candidates, f)
with open('/content/drive/MyDrive/pickledData/btest_lhs_candidates.pickle', 'rb') as f:
  adult_lhs_candidates = pickle.load(f)

#print(adult_lhs_candidates)
#print(len(adult_lhs_candidates))
#print("LHS candidates",adult_lhs_candidates)

# save all tuples of length 4 into a set
newRFDCandidates = set()
def getrfds(lhs):
  for set in adult_lhs_candidates:
    for tuple in set:
       if len(tuple) ==4:
        newRFDCandidates.add(tuple)
  return newRFDCandidates

#lhs_len_two = getrfds(adult_lhs_candidates)

# determine thresholds of viable candidates in lhs_len_four
def all_thresholds(TB_relation, LHS):
  all_RFDcs = []

  for w in TB_relation:
    RFDcs_for_w = determine_thresholds(TB_relation, w, LHS)
    all_RFDcs.append(RFDcs_for_w)

  return all_RFDcs

#all_RFDcs_lentwo = all_thresholds(sample_vectors,lhs_len_two)


with open('/content/drive/MyDrive/pickledData/filteredRfdsLenTwo.pickle', 'rb') as f:
   filtered_rfds_len_two = pickle.load(f)

with open('/content/drive/MyDrive/pickledData/adultSetRfdsLenthree.pickle', 'rb') as f:
  adult_set_rfds_len_three = pickle.load(f)

with open('/content/drive/MyDrive/pickledData/adultSetRfdsLenfour.pickle', 'rb') as f:
  adult_set_rfds_len_four = pickle.load(f)

#print("RFDS",all_RFDcs_lentwo)
#print(len(all_RFDcs))
#print(all_RFDcs[1])
#print(len(all_RFDcs[1]))

def filter_rfds(list_of_rfds, index_to_remove):
  for rfd in list_of_rfds:
    if index_to_remove in rfd["cc"]:
      list_of_rfds.remove(rfd)
  return list_of_rfds

sample_rfd_len_four = random.sample(adult_set_rfds_len_four, 100)
print(sample_rfd_len_four)
sample_rfd_len_three = random.sample(adult_set_rfds_len_three, 200)
print(sample_rfd_len_three)

[{'thresholds': [1, 1, 1, 1], 'cc': (1, 2, 10, 11)}, {'thresholds': [2, 1, 1, 4], 'cc': (2, 4, 5, 10)}, {'thresholds': [4, 2, 1, 4], 'cc': (0, 2, 3, 8)}, {'thresholds': [1, 1, 1, 1], 'cc': (1, 2, 4, 5)}, {'thresholds': [2, 1, 4, 1], 'cc': (2, 3, 10, 11)}, {'thresholds': [3, 1, 1, 4], 'cc': (0, 1, 5, 10)}, {'thresholds': [4, 1, 1, 1], 'cc': (0, 2, 3, 11)}, {'thresholds': [2, 1, 1, 4], 'cc': (2, 3, 5, 10)}, {'thresholds': [4, 1, 1, 4], 'cc': (0, 1, 5, 8)}, {'thresholds': [4, 1, 1, 1], 'cc': (0, 1, 2, 4)}, {'thresholds': [2, 1, 1, 1], 'cc': (2, 3, 5, 9)}, {'thresholds': [4, 2, 1, 1], 'cc': (0, 2, 3, 6)}, {'thresholds': [4, 2, 4, 1], 'cc': (0, 2, 10, 11)}, {'thresholds': [4, 1, 1, 1], 'cc': (0, 1, 2, 11)}, {'thresholds': [4, 1, 1, 1], 'cc': (0, 2, 3, 4)}, {'thresholds': [2, 1, 1, 4], 'cc': (2, 3, 5, 10)}, {'thresholds': [1, 1, 1, 1], 'cc': (1, 2, 5, 9)}, {'thresholds': [1, 3, 4, 1], 'cc': (1, 8, 10, 11)}, {'thresholds': [1, 1, 1, 1], 'cc': (1, 2, 3, 4)}, {'thresholds': [1, 2, 1, 1], 'cc': 

#RFD join algorithm

The following code handles the anonymization using RFDs as proposed in the paper "A decision-support framework for data anonymization with
application to machine learning processes", by L.Caruccio et. al.



```
@article{CARUCCIO20221,
title = {A decision-support framework for data anonymization with application to machine learning processes},
journal = {Information Sciences},
volume = {613},
pages = {1-32},
year = {2022},
issn = {0020-0255},
doi = {https://doi.org/10.1016/j.ins.2022.09.004},
url = {https://www.sciencedirect.com/science/article/pii/S0020025522010490},
author = {Loredana Caruccio and Domenico Desiato and Giuseppe Polese and Genoveffa Tortora and Nicola Zannone},
keywords = {Privacy preserving machine learning, k-anonymity, Relaxed functional dependencies, Generalization strategies}
}
```



In [ ]:
qis = ["age", "workclass", "fnlwgt", "education", "marital-status", "occupation",
                "relationship", "sex","capital-gain", "capital-loss", "hours-per-week",
                "native-country"]


# find all equivalence classes in a dataset and label them
def find_equivalence_classes(dataset, groupby_columns):
    dataset_with_classes = dataset.copy()
    # new column tracks the equivalence classes
    dataset_with_classes['EquivalenceClass'] = dataset_with_classes.groupby(groupby_columns).ngroup()

    return dataset_with_classes

# find equivalence classes with a specified RFD
def find_equivalence_classes_by_rfd(dataset, rfd):
    dataset_copy = dataset.copy()
    cc = rfd.get("cc")
    attributes = []

    for value in cc:
      attribute = qis[value]
      attributes.append(attribute)

    dataset_with_classes = dataset.copy()

    dataset_with_classes['EquivalenceClass'] = dataset_with_classes.groupby(attributes).ngroup()

    return dataset_with_classes

# filter dataset to get rows belonging to the specified equivalence class ID
def get_equivalence_class_by_id(df, eq_class_id):
    equivalence_class = df[df['EquivalenceClass'] == eq_class_id]
    return equivalence_class

# get frequency of each equivalence class
def equivalence_class_sizes(dataset_with_classes):
    class_sizes = dataset_with_classes.groupby('EquivalenceClass').size()
    size_frequencies = class_sizes.value_counts().sort_values(ascending=False).to_dict()

    return size_frequencies



# compute anonymization level for k-anonymity
def compute_k(dataset):
  groups = find_equivalence_classes(dataset, qis)
  k = 100
  for ec_id, group in groups.groupby('EquivalenceClass'):
    if len(group) < k:
      k = len(group)

  return k

# generalize numeric attribute to specified level
def generalize_by_threshold_numeric(dataset, attribute, genLevel):

  hierarchy = check_hierarchy(attribute)

  def gen_by_genLevel(hierarchy, x):
    gen_interval = ()

    for interval in hierarchy[genLevel-1]:
      if x <= interval[1] and x >= interval[0]:
        gen_interval = interval
        break
    return gen_interval

  dataset[attribute] = dataset[attribute].apply(lambda x: gen_by_genLevel(hierarchy, x))
  return dataset

# generalize categoric attribute to specified level
def generalize_by_threshold_categoric(dataset, attribute, genLevel):
  hierarchy = check_hierarchy(attribute)

  if genLevel >1:
    dataset[attribute] = dataset[attribute].apply(lambda x: list(hierarchy.keys())[0])
  else:
    for key, value in hierarchy.items():
      if isinstance(value, dict):
        for inner_key, inner_value in value.items():
          dataset[attribute] = dataset[attribute].apply(lambda x: inner_key if x in inner_value else x)

  return dataset

# generalize dataset with given RFD

def generalize_by_rfd(dataset, rfd):
  dataset_copy = dataset.copy()
  cc = rfd.get("cc")
  thrs = rfd.get("thresholds")
  attributes = []

  for value in cc:
    attribute = qis[value]
    attributes.append(attribute)

  for attribute in attributes:
    if dataset_copy[attribute].dtype == "int64":
      generalize_by_threshold_numeric(dataset_copy, attribute, thrs[attributes.index(attribute)])
    else:
      generalize_by_threshold_categoric(dataset_copy, attribute, thrs[attributes.index(attribute)])

  all_columns = dataset_copy.columns

    # Iterate over columns not in 'attributes' and replace with '*'
  for column in all_columns:
    if column not in attributes and column != "income":
      dataset_copy[column] = '*'

  dataset_copy['EquivalenceClass'] = dataset_copy.groupby(attributes).ngroup()

  return dataset_copy



# check, if two attributes are generalized to the same level in two seperate RFDs
def has_equal_level(x,y):
  cc_x = x.get("cc")
  cc_y = y.get("cc")

  thrs_x = x.get("thresholds")
  thrs_y = y.get("thresholds")

  c = set(cc_x).intersection(cc_y)

  equal_level = True

  for value in c:
    if value in cc_x and value in cc_y:
      #if thrs_x[cc_x.index(value)] != thrs_y[cc_y.index(value)]:
      index_x = cc_x.index(value)
      index_y = cc_y.index(value)
      if index_x < len(thrs_x) and index_y < len(thrs_y) and thrs_x[index_x] != thrs_y[index_y]:
        equal_level = False
        break
    else:
      equal_level = False
      break
  return equal_level



# calculate the NCP of a numeric value
def ncp_numeric(dataset, genDataset, attribute):
  interval = genDataset[attribute].iloc[0]
  if interval == "*":
    return 1
  else:
    if isinstance(interval, str) and interval.startswith('(') and interval.endswith(')'):

      interval = eval(interval)

    lower, upper = interval[0], interval[1] if isinstance(interval, tuple) else (interval, interval)
    minVal = dataset[attribute].min()
    maxVal = dataset[attribute].max()

  return (upper - lower) / (maxVal - minVal)

# compute the NCP of a categorical value
def ncp_categorical(dataset, genDataset, attribute):
  att_hierarchy = check_hierarchy(attribute)

  ncp = 0
  gen_value = genDataset[attribute].iloc[0]

  if gen_value == "*":
    return 1
  else:
    for key, value in att_hierarchy.items():
      if gen_value == key:
        ncp = 1
      elif isinstance(value, dict) and gen_value in value.keys():
        ncp = len(value.keys()) / len(dataset[attribute].unique())
        break
  return ncp

# check datatype of attribute
def kind_of_attribute(attribute):
  if attribute in ["age", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week"]:
    return "numeric"
  else:
    return "categorical"


# compute the NCP of an equivalence class
def compute_ncp(dataset, genDataset, equivalenceClass):
  all_columns = dataset.columns.tolist()
  attributes = [col for col in all_columns if col != 'income']
  ncp = 0
  m = len(attributes)
  for attribute in attributes:
    if kind_of_attribute(attribute) == "numeric":
      ncp += ncp_numeric(dataset, equivalenceClass, attribute)
    else:
      ncp += ncp_categorical(dataset, equivalenceClass, attribute)
  return ncp / m

# compute the NCP of the whole dataset
def compute_gncp(dataset,genDataset):
  gncp = 0
  for eq_class_id, group in genDataset.groupby('EquivalenceClass'):
    gncp += (len(group)/ len(dataset)) * compute_ncp(dataset,genDataset, group)
  return gncp

# merge two RFDs
def rfd_merge(rfd1, rfd2):
    combined_cc = tuple(set(rfd1['cc'] + rfd2['cc']))
    threshold_map = {}
    for i, val in enumerate(rfd1['cc']):
        if i < len(rfd1['thresholds']):
            threshold_map[val] = rfd1['thresholds'][i]
    for i, val in enumerate(rfd2['cc']):
        if i < len(rfd2['thresholds']) and val not in threshold_map:
            threshold_map[val] = rfd2['thresholds'][i]

    if all(val in threshold_map for val in combined_cc):
        merged_thresholds = [threshold_map[val] for val in combined_cc]
        merged_set = {'thresholds': merged_thresholds, 'cc': combined_cc}
        return merged_set
    else:
        return None

# merge two RFDs with different sets of column combinations
def check_compatibility(rfdsOne, rfdsTwo):
  compatible_rfds = []
  for i, xi in enumerate(rfdsOne):
    for j, yi in enumerate(rfdsTwo):
      if i >= j:
        continue
      X = set(xi.get("cc"))
      Y = set(yi.get("cc"))
      c = X.intersection(Y)
      # Check compatibility of xi and yi
      if X.isdisjoint(Y): #or has_equal_level(xi,yi):
        ci = rfd_merge(xi,yi)
        if ci is not None:
          compatible_rfds.append(ci)
  return compatible_rfds

#compatible_rfds_len_eight = check_compatibility(sample_rfd_len_four)

with open('/content/drive/MyDrive/pickledData/rfdsLenEight.pickle', 'rb') as f:
  compatible_rfds_len_eight = pickle.load(f)

rfds_len_eight_and_three = []

#for rfd in sample_rfd_len_three:
  #rfds_len_eight_and_three.append(rfd)

#for rfd in compatible_rfds_len_eight:
#  rfds_len_eight_and_three.append(rfd)

#print(len(rfds_len_eight_and_three))

#compatible_rfds_len_eight_and_three = check_compatibility(rfds_len_eight_and_three)


with open('/content/drive/MyDrive/pickledData/rfdsLenEightandThree.pickle', 'rb') as f:
  compatible_rfds_len_eight_and_three = pickle.load(f)

#print(len(compatible_rfds_len_eight_and_three))
#for x in range(50):
# print(compatible_rfds_len_eight_and_three[0])


def get_unique_rfds(rfds):
    unique_rfds = []
    seen_rfds = set()

    def rfd_to_tuple(rfd):
        """Converts an RFD dictionary to a tuple for hashable representation."""
        return (tuple(rfd.get('cc', ())), tuple(rfd.get('thresholds', ())))

    for rfd in rfds:
        rfd_tuple = rfd_to_tuple(rfd)
        if rfd_tuple not in seen_rfds:
            seen_rfds.add(rfd_tuple)
            unique_rfds.append(rfd)
            #if len(unique_rfds) == len(rfds):
             #   break  # Stop after collecting 100 unique RFDs
    return unique_rfds

# Get 200 unique RFDs
#unique_rfds_200 = get_200_unique_rfds(compatible_rfds_len_eight_and_three)
#print(unique_rfds_200)

with open('/content/drive/MyDrive/pickledData/200uniqueRfds.pickle', 'rb') as f:
  unique_rfds_200 = pickle.load(f)

#unique_rfds_200_with_len_four = check_compatibility(unique_rfds_200, sample_rfd_len_four)


with open('/content/drive/MyDrive/pickledData/200uniqueRfdsandLenFour.pickle', 'rb') as f:
  unique_rfds_200_with_len_four = pickle.load(f)

#final_rfds = get_unique_rfds(unique_rfds_200_with_len_four)


with open('/content/drive/MyDrive/pickledData/finalRFds.pickle', 'rb') as f:
  final_rfds = pickle.load(f)

rfds_with_info = []
#for rfd in final_rfds:
 # r = ()
 # result = generalize_by_rfd(adult, rfd)
 # result_with_eq_class = find_equivalence_classes_by_rfd(result, rfd)
 # k_level =  equivalence_class_sizes(result_with_eq_class)
 # m = compute_gncp(adult, result_with_eq_class)
 # r = (rfd, m, k_level)
 # rfds_with_info.append(r)



with open('/content/drive/MyDrive/pickledData/Rfdswithinfo.pickle', 'rb') as f:
  rfds_with_info = pickle.load(f)




# find matching equivalence classes for a list of equivalence classes
def find_matching_equivalence_classes(dataset, equivalence_class_numbers, k_anonymity_value):
    matching_columns =["age", "workclass", "fnlwgt", "hours-per-week"]
    matching_classes = {}
    minimal_matching_columns = ["age", "workclass"]  # Define the minimal columns here

    for eq_class_num in equivalence_class_numbers:
        matching_classes[eq_class_num] = []
        current_eq_class_size = len(dataset[dataset['EquivalenceClass'] == eq_class_num])
        eq_class_data = dataset[dataset['EquivalenceClass'] == eq_class_num][matching_columns].iloc[0]

        # 1. Prioritize matches within equivalence_class_numbers using matching_columns
        for other_eq_class_num in equivalence_class_numbers:
            if other_eq_class_num != eq_class_num:
                other_eq_class_data = dataset[dataset['EquivalenceClass'] == other_eq_class_num][matching_columns].iloc[0]
                other_eq_class_size = len(dataset[dataset['EquivalenceClass'] == other_eq_class_num])

                if eq_class_data.equals(other_eq_class_data) and other_eq_class_size >= (k_anonymity_value - current_eq_class_size):
                    matching_classes[eq_class_num].append(other_eq_class_num)
                    if len(matching_classes[eq_class_num]) >= 2:
                        break

        # 2. Search entire dataset using minimal_matching_columns if no matches found within
        if not matching_classes[eq_class_num]:
            eq_class_data_minimal = dataset[dataset['EquivalenceClass'] == eq_class_num][minimal_matching_columns].iloc[0]
            for other_eq_class_num in dataset['EquivalenceClass'].unique():
                if other_eq_class_num not in equivalence_class_numbers:  # Exclude those already checked
                    other_eq_class_data_minimal = dataset[dataset['EquivalenceClass'] == other_eq_class_num][minimal_matching_columns].iloc[0]
                    other_eq_class_size = len(dataset[dataset['EquivalenceClass'] == other_eq_class_num])

                    if eq_class_data_minimal.equals(other_eq_class_data_minimal) and other_eq_class_size >= (k_anonymity_value - current_eq_class_size):
                        matching_classes[eq_class_num].append(other_eq_class_num)
                        if len(matching_classes[eq_class_num]) >= 2:
                            break

    return matching_classes


def find_matching_l_diverse_classes(dataset, target_eq_class_id, exclude_eq_class_ids, l_diversity_value):

    matching_columns = ["age", "fnlwgt"]  # Attributes to match on
    sensitive_attribute = "income"  # Sensitive attribute for l-diversity check
    matching_classes = []

    # Get data for the target equivalence class
    target_eq_class_data = dataset[dataset['EquivalenceClass'] == target_eq_class_id][matching_columns].iloc[0]
    target_eq_class_size = len(dataset[dataset['EquivalenceClass'] == target_eq_class_id])

    # Get all unique equivalence class IDs except those in exclude_eq_class_ids
    all_eq_class_ids = dataset['EquivalenceClass'].unique()
    eq_class_ids_to_search = [eq_id for eq_id in all_eq_class_ids if eq_id not in exclude_eq_class_ids and eq_id != target_eq_class_id]

    # Search for matching l-diverse classes
    for eq_class_id in eq_class_ids_to_search:
        # Check for matching attributes
        eq_class_data = dataset[dataset['EquivalenceClass'] == eq_class_id][matching_columns].iloc[0]
        eq_class_size = len(dataset[dataset['EquivalenceClass'] == eq_class_id])

        # Check if class size is at least k - len(current equality class) and if it is l-diverse
        if target_eq_class_data.equals(eq_class_data) and is_l_diverse(dataset[dataset['EquivalenceClass'] == eq_class_id], sensitive_attribute, l_diversity_value):
            matching_classes.append(eq_class_id)

    return matching_classes

def find_matching_k_classes(dataset, target_eq_class_id, exclude_eq_class_ids, k_anonymity_value):

    matching_columns = ["age", "fnlwgt"]  # Attributes to match on
    sensitive_attribute = "income"  # Sensitive attribute for l-diversity check
    matching_classes = []

    # Get data for the target equivalence class
    target_eq_class_data = dataset[dataset['EquivalenceClass'] == target_eq_class_id][matching_columns].iloc[0]
    target_eq_class_size = len(dataset[dataset['EquivalenceClass'] == target_eq_class_id])

    # Get all unique equivalence class IDs except those in exclude_eq_class_ids
    all_eq_class_ids = dataset['EquivalenceClass'].unique()
    eq_class_ids_to_search = [eq_id for eq_id in all_eq_class_ids if eq_id not in exclude_eq_class_ids and eq_id != target_eq_class_id]

    # Search for matching l-diverse classes
    for eq_class_id in eq_class_ids_to_search:
        # Check for matching attributes
        eq_class_data = dataset[dataset['EquivalenceClass'] == eq_class_id][matching_columns].iloc[0]
        eq_class_size = len(dataset[dataset['EquivalenceClass'] == eq_class_id])

        # Check if class size is at least k - len(current equality class) and if it is l-diverse
        if target_eq_class_data.equals(eq_class_data) and eq_class_size >= (k_anonymity_value - target_eq_class_size):
            matching_classes.append(eq_class_id)

    return matching_classes


def find_not_k_anonymous_eq_class(dataset, k):
  not_k_anonymous_classes = []
  for eq_class_id, group in dataset.groupby('EquivalenceClass'):
    if len(group) < k:
      #print(f"\nEquivalence Class {eq_class_id}:\n{group}")
      #print(len(group))
      not_k_anonymous_classes.append(eq_class_id)
  return not_k_anonymous_classes



def anonymize_equivalence_classes(dataset, equivalence_class_ids):
    anonymized_dataset = dataset.copy()

    columns_to_anonymize = [col for col in anonymized_dataset.columns if col != 'EquivalenceClass']
    for eq_class_id in equivalence_class_ids:
        # Filter the dataset for the current equivalence class
        equivalence_class_data = anonymized_dataset[anonymized_dataset['EquivalenceClass'] == eq_class_id]

        # Replace values in the specified columns with '*'
        for column in columns_to_anonymize:
            anonymized_dataset.loc[equivalence_class_data.index, column] = '*'

    return anonymized_dataset

# generalize intervals for the merge operation of two equivalence classes
def generalize_intervals(hierarchy, intervalOne, intervalTwo, genLevel):
  gen_interval = ()

  for interval in hierarchy[genLevel-1]:
    if intervalTwo[1] <= interval[1] and intervalOne[0] >= interval[0]:
      gen_interval = interval
      break
    elif intervalOne[1] <= interval[1] and intervalTwo[0] >= interval[0]:
      gen_interval = interval
      break
  return gen_interval


def merge_equivalence_classes(dataset, not_k_anon_class_id, k_anon_class_id):

    # Get the rows for the two equivalence classes
    eq_class1_rows = dataset[dataset['EquivalenceClass'] == not_k_anon_class_id]
    eq_class2_rows = dataset[dataset['EquivalenceClass'] == k_anon_class_id]

    # Get a sample row from each equivalence class for modification
    sample_row1 = eq_class1_rows.iloc[0].copy()
    sample_row2 = eq_class2_rows.iloc[0].copy()

    # Iterate over columns, generalize values if not in exclude_columns
    for column in dataset.columns:
        if column not in ["income", 'EquivalenceClass']:  # Exclude EquivalenceClass column
            value1 = sample_row1[column]
            value2 = sample_row2[column]

            if value1 == value2:
              continue

            elif kind_of_attribute(column) == "numeric" and value1 != "*":
              left = min(value1[0], value2[0])
              right = max(value1[0], value2[0])

              sample_row1[column] = (left,right)
              sample_row2[column] = (left,right)
            elif kind_of_attribute(column) == "categorical":

              sample_row1[column] = "*"
              sample_row2[column] = "*"

    for column in dataset.columns:
        if column not in ["income", 'EquivalenceClass']:
            dataset.loc[dataset['EquivalenceClass'] == not_k_anon_class_id, column] = str(sample_row1[column])
            dataset.loc[dataset['EquivalenceClass'] == k_anon_class_id, column] = str(sample_row2[column])
    # Merge equivalence classes by changing eq_class_id2 to eq_class_id1
    dataset.loc[dataset['EquivalenceClass'] == not_k_anon_class_id, 'EquivalenceClass'] = k_anon_class_id

    return dataset



# make sure the dataset is l-diverse
def achieve_l_diversity(dataset, sensitive_attribute, l_diversity_value):


    previous_num_not_l_diverse = float('inf')

    while True:
        not_l_diverse_classes = []
        for eq_class_id, group in dataset.groupby('EquivalenceClass'):
            if not is_l_diverse(group, sensitive_attribute, l_diversity_value):
                not_l_diverse_classes.append(eq_class_id)

        if not not_l_diverse_classes:  # If all classes are l-diverse, exit
            break

        # Check if the number of non-l-diverse classes has not decreased
        if len(not_l_diverse_classes) == previous_num_not_l_diverse:
            print("Warning: Cannot achieve l-diversity with current matching criteria.")
            break

        previous_num_not_l_diverse = len(not_l_diverse_classes)

        #matching_classes_dict = find_matching_equivalence_classes(dataset, not_l_diverse_classes, k_anonymity_value)

        for eq_class_id in not_l_diverse_classes:
          found_matches = find_matching_l_diverse_classes(dataset, eq_class_id, not_l_diverse_classes, l_diversity_value)
            #matches = matching_classes_dict.get(eq_class_id, [])

            # Filter matches to exclude classes in not_l_diverse_classes
            #valid_matches = [match for match in matches if match not in not_l_diverse_classes]

          if found_matches !=[]:
            merge_with = min(found_matches)  # Choose the smallest valid matching class ID
            dataset = merge_equivalence_classes(dataset, eq_class_id, merge_with)
          elif found_matches == []:
            continue
    return dataset

#l_div_5_anon_adult = achieve_l_diversity(sevenfive_anonymized_dataset, "income",2)

with open('/content/drive/MyDrive/pickledData/RFDl_div_5_anon_adult.pickle', 'rb') as f:
  l_div_100_anon_adult = pickle.load(f)





0.6294084033329445


In [ ]:
# Apply above functions to the obesity dataset



# create hierarchies
CAEC_hierarchy = {
    "*": {
        "Does Not Eat Between Meals": ["no"],
        "Eats Between Meals": ["Sometimes", "Frequently", "Always"]
    }
}

CALC_hierarchy = {
    "*": {
        "does not drink alcohol": ["no"],
        "drinks alcohol": ["Sometimes", "Frequently", "Always"]
    }
}

MTRANS_hierarchy = {
    "*": {
        "Motorized Transportation": ["Public_Transportation", "Automobile", "Motorbike"],
        "Non-Motorized Transportation": ["Walking", "Bike"]
    }
}

nobeyesdad_hierarchy ={"NObeyesdad": ['Normal_Weight' 'Overweight_Level_I' 'Overweight_Level_II'
 'Obesity_Type_I' 'Insufficient_Weight' 'Obesity_Type_II'
 'Obesity_Type_III']}

# fam_history_with_overweight, FAVC, SMOKE, SCC
binary_hierarchy = {"*": ["yes", "no"]}

gender_hierarchy = {"*": ["Male", "Female"]}



def generalize_numeric_obesity(dataset, qi):
  min_val, max_val = dataset[qi].min(), dataset[qi].max()
  if min_val == max_val:
    dataset[qi] = "*"
  elif qi == "Age":
    min_val = min_val - min_val % 5
    max_val = max_val + 5 - (max_val % 5)
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "Height":
    min_val = math.floor(min_val)
    max_val = math.ceil(max_val)
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "Weight":
    min_val = min_val - min_val % 5
    max_val = max_val + 5 - (max_val % 5)
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "FCVC":
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "NCP":
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "CH2O":
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "FAF":
    dataset[qi] = f"[{min_val} - {max_val}]"
  elif qi == "TUE":
    dataset[qi] = f"[{min_val} - {max_val}]"
  return dataset

FCVC_hierarchy = [[(1.5,1), (1.5,2), (2,2,5), (2.5,3)],[(1,2),(2,3)], [(1,3)]]
NCP_hierarchy = [[(1,1.5), (1.5,2), (2,2.5), (2.5,3), (3.5,4)],[(1,2),(2,3),(3,4)],[(1,2),(2,4)], [(1,4)]]
height_hierarchy = [[(1,1.5),(1.5,2)], [(1,2)]]
tue_hierarchy = [[(0.0, 0.5), (0.5, 1.0), (1.0, 1.5), (1.5, 2.0)], [(0.0, 1.0), (1.0, 2.0)], [(0.0, 2.0)]]
CH2O_hierarchy = [[(1.5,1), (1.5,2), (2.5,3)],[(1,2),(2,3)], [(1,3)]]
FAF_hierarchy = [[(0,0.5),(0.5,1), (1,1.5), (1.5,2),(2,2.5), (2.5,3)], [(0,1), (1,2), (2,3)], [(0,2), (2,3)], [(0,3)]]
weight_values = obesityDataset["Weight"].unique()
weight_hierarchy = create_gen_hierarchy(weight_values, 5)
#print(weight_hierarchy)
age_obesity_values = obesityDataset["Age"].unique()
age_obesity_hierarchy = create_gen_hierarchy(age_obesity_values, 5)
#print(age_obesity_hierarchy)
weight_values = obesityDataset["Weight"].unique()
weight_hierarchy = create_gen_hierarchy(weight_values, 5)
#print(weight_hierarchy)
#print(age_obesity_hierarchy)
nobeyesdad_obesity_values = obesityDataset["NObeyesdad"].unique()



def check_obesity_hierarchy(attribute):
  if attribute == 'Gender':
    return gender_hierarchy
  elif attribute == 'Age':
    return age_obesity_hierarchy
  elif attribute == 'Height':
    return height_hierarchy
  elif attribute == 'Weight':
    return weight_hierarchy
  elif attribute == 'FCVC':
    return FCVC_hierarchy
  elif attribute == 'NCP':
    return NCP_hierarchy
  elif attribute == "CAEC":
    return CAEC_hierarchy
  elif attribute == "CH2O":
    return CH2O_hierarchy
  elif attribute == "FAF":
    return FAF_hierarchy
  elif attribute == "TUE":
    return tue_hierarchy
  elif attribute == "CALC":
    return CALC_hierarchy
  elif attribute == "MTRANS":
    return MTRANS_hierarchy
  elif attribute == "NObeyesdad":
    return nobeyesdad_hierarchy
  else:
    return binary_hierarchy


def is_k_anonymous_categorical(partition, categorical_qis):

    # Select only the categorical quasi-identifier columns
    qi_data = partition[categorical_qis]

    # Count the occurrences of unique combinations of categorical QI values
    counts = qi_data.groupby(categorical_qis).size()

    # Check if all counts are greater than or equal to k
    # and explicitly return True or False
    k = len(partition)
    if (counts >= k).all():
        return True
    else:
        return False

def refine_generalization(dataset, qi):
# Work on a copy to avoid modifying the original

    #for qi in qis:
      #  if kind_of_attribute(qi) == "categorical":  # Numeric or categorical that has been generalized

  hierarchy = check_obesity_hierarchy(qi)
  if isinstance(hierarchy, list):
    print(f"Attribute: {qi}, Hierarchy Type: {type(hierarchy)}")
  if hierarchy is not None and isinstance(hierarchy, dict):
    dataset[qi] = dataset[qi].apply(lambda x: gen_to_higher_level(x, hierarchy))

  return dataset

# make partition k-anonymous
def ensure_k_anonymity_categorical(partition):
  categorical_qis = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE','SCC', 'CAEC', 'CALC', 'MTRANS']
  #["workclass", "education", "marital-status", "relationship", "occupation", "native-country"]

  generalized_partition = partition.copy()
  while not is_k_anonymous_categorical(generalized_partition, categorical_qis):
        # Find categorical columns with multiple values
        columns_to_generalize = [
            col for col in categorical_qis
            if generalized_partition[col].nunique() > 1  # More than one unique value
        ]

        if not columns_to_generalize:
            # If no columns can be generalized further, break the loop
            break

        # Generalize the first column in the list (minimal generalization)
        column_to_generalize = columns_to_generalize[0]
        print(column_to_generalize)
        hierarchy = check_obesity_hierarchy(column_to_generalize)

        generalized_partition[column_to_generalize] = generalized_partition[column_to_generalize].apply(lambda x: gen_to_higher_level(x, hierarchy))


  return generalized_partition






def generalize(dataset,qis):
  for qi in qis:
    if kind_of_attribute(qi) == "numeric":
      dataset = generalize_numeric_obesity(dataset, qi)
    else:
      dataset = refine_generalization(dataset, qi)

  dataset = ensure_k_anonymity_categorical(dataset)
  return dataset


"""
This recursive function checks for k-anonymity and l-diversity in each recursion
step. The algorithm is based on the algorithm of LeFevre et. al., as stated above.
"""
def l_mondrian(dataset, k, qis, l, sensitive_attribute):
    global recursive_call



    # Base cases for recursion
    #print(len(dataset))
    #print(entropy_partition(dataset, sensitive_attribute))
    if len(dataset) < 2 * k:
      return generalize(dataset.copy(), qis)  # Use a copy to ensure isolation
    if recursive_call == 0:
      selected_qi = "Age"
    else:
      selected_qi = widest_qi(dataset, qis)

    print(selected_qi)
    #selected_qi = widest_qi(dataset, qis)  # choose QI with the most unique values
    #print(f"Selected QI: {selected_qi}")  # Debug print

    # Handle numeric attributes
    if dataset[selected_qi].dtype == np.float64:   #"int64"
        sorted_dataset = dataset.sort_values(by=selected_qi).copy()  # Ensure a copy for safety
        middle_index = len(sorted_dataset) // 2
        left_side = sorted_dataset.iloc[:middle_index].copy()
        right_side = sorted_dataset.iloc[middle_index:].copy()

    # Handle categorical attributes
    elif dataset[selected_qi].dtype == object:
        print(selected_qi)
        hierarchy = check_obesity_hierarchy(selected_qi)

        dataset[selected_qi] = dataset[selected_qi].apply(lambda x: gen_to_higher_level(x, hierarchy))  # Isolate generalized data
        sorted_partition = sorted(dataset[selected_qi].unique())
        split_value = find_middle(sorted_partition)
        group_left = sorted_partition[:sorted_partition.index(split_value)]
        group_right = sorted_partition[sorted_partition.index(split_value):]
        left_side = dataset[dataset[selected_qi].isin(group_left)].copy()
        right_side = dataset[dataset[selected_qi].isin(group_right)].copy()

    # Recursively apply l_mondrian on the partitions
    if (len(left_side) >=k and is_l_diverse(left_side, sensitive_attribute, l)) and (len(right_side) >= k and is_l_diverse(right_side, sensitive_attribute, l)):
      recursive_call += 1
      partition_one = l_mondrian(left_side, k, qis, l, sensitive_attribute)
      partition_two = l_mondrian(right_side, k, qis, l, sensitive_attribute)


    else:
      return generalize(dataset.copy(), qis)


    # Concatenate the generalized partitions
    return pd.concat([partition_one, partition_two])



distance_data_obesity = []

# test L_mondrian
#qi_columns= ["Gender", "Age", "Height", "Weight","family_history_with_overweight","FAVC", "FCVC", "NCP", "CAEC", "SMOKE", "CH2O","SCC", "FAF", "TUE", "CALC", "MTRANS","NObeyesdad"]
#nine_mondrian_obesity_50_anon = l_mondrian(obesityDataset, 10,['Age','Height', 'Weight','FCVC','NCP', 'CH2O', 'FAF', 'TUE','MTRANS'],9 ,"NObeyesdad")
#nine_mondrian_obesity_10_anon_eq = find_equivalence_classes(nine_mondrian_obesity_50_anon, qi_columns)
#display(nine_mondrian_obesity_five_anon)


#with open('/content/drive/MyDrive/pickledData/nine_mondrian_obesity_10_anon_eq.pickle', 'wb') as f:
 #   pickle.dump(nine_mondrian_obesity_10_anon_eq, f)
#with open('/content/drive/MyDrive/pickledData/nine_mondrian_obesity_75_anon_eq.pickle', 'rb') as f:
#    nine_mondrian_obesity_100_anon_eq = pickle.load(f)

# create distance matrix with generalization distance as metric
def compute_delta_df(dataset, num_rows, qi_columns, sensitive_attribute):
  for i in range(num_rows):
    for j in range(i + 1, num_rows):
        row_distances = {}
        for column in dataset.columns:
            if column in qi_columns:
                if dataset[column].dtype == "float64":
                    distance = generalization_distance_numeric(dataset.loc[i, column], dataset.loc[j, column], check_obesity_hierarchy(column))
                else:
                    distance = generalization_distance_categoric(dataset.loc[i, column], dataset.loc[j, column], check_obesity_hierarchy(column))
                row_distances[column] = distance
            else:
                row_distances[column] = None
        distance_data_obesity.append(row_distances)
  distance_df = pd.DataFrame(distance_data_obesity)
  return distance_df.sort_values(by=sensitive_attribute)

#testDelta = compute_delta_df(testDM_sample, test_rows, test_columns, "income")

obesity_qi_columns= ["Gender", "Age", "Height", "Weight","family_history_with_overweight","FAVC", "FCVC", "NCP", "CAEC", "SMOKE", "CH2O","SCC", "FAF", "TUE", "CALC", "MTRANS","NObeyesdad"]
#obesity_sample = obesityDataset.head(150)

#obesity_delta = compute_delta_df(obesity_sample, 150, obesity_qi_columns, "NObeyesdad")
#len(obesity_delta)
#display(obesity_delta)



# create T_beta
#t_zero_obesity = create_T_relations(obesity_delta, "NObeyesdad")

with open('/content/drive/MyDrive/pickledData/btzeroObesity.pickle', 'rb') as f:
    t_zero_obesity = pickle.load(f)




#obesity_lhs_candidates = all_lhs_tbeta(t_zero_obesity[0])


with open('/content/drive/MyDrive/pickledData/obesity_lhs_candidates.pickle', 'rb') as f:
  obesity_lhs_candidates = pickle.load(f)


newRFDCandidates_obesity = set()
#def getrfds(lhs):
 # for set in obesity_lhs_candidates:
  #  for tuple in set:
   #    if len(tuple) ==3:
    #    newRFDCandidates_obesity.add(tuple)
 # return newRFDCandidates_obesity

#lhs_len_three_obesity = getrfds(obesity_lhs_candidates)

#all_rfds_len_three_obesity = all_thresholds(t_zero_obesity[0],lhs_len_three_obesity)

#with open('/content/drive/MyDrive/pickledData/obesityLenThreeRFDs.pickle', 'wb') as f:
 #   pickle.dump(all_rfds_len_three_obesity, f)
#with open('/content/drive/MyDrive/pickledData/obesityLenThreeRFDs.pickle', 'rb') as f:
 # all_rfds_len_three_obesity = pickle.load(f)
with open('/content/drive/MyDrive/pickledData/obesityLenFourRFDs.pickle', 'rb') as f:
  all_rfds_len_four_obesity = pickle.load(f)

print(all_rfds_len_four_obesity)

#obesity_comatible_rfds_len_eight = check_compatibility(all_rfds_len_four_obesity, all_rfds_len_four_obesity)
#print(obesity_comatible_rfds_len_eight)

#with open('/content/drive/MyDrive/pickledData/obesity_comatible_rfds_len_eight.pickle', 'wb') as f:
 #   pickle.dump(obesity_comatible_rfds_len_eight, f)

In [ ]:
adultresult = pd.read